# Prepare data for a analysis using CMS tools

- Baseando-se, entre outras coisas, no analysis grand challange: https://cms-opendata-workshop.github.io/workshop2024-lesson-remote-hackathon/06-agc.html


In [9]:
# First, install dependecies
!pip install -r ../requirements.txt


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [10]:
!pip install --upgrade pip

!pip install --upgrade awkward
!pip install --upgrade uproot
!pip install fsspec-xrootd
!pip install vector
!pip install --upgrade pandas
!pip install --upgrade coffea
!pip install --upgrade matplotlib

  Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.7/831.7 kB 6.9 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.1/633.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: awkward
    Found existing installation: awkward 1.10.3
    Uninstalling awkward-1.10.3:
      Successfully uninstalled awkward-1.10.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
coffea 0.7.21 requires awkward<2,>=1.10.3, but you have awkward 2.6.8 which is incompatible.
servicex 2.6.2 requires awkward<2,>=1.0.1, but you have awkward 2.6.8 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: uproot
    Found e

In [ ]:
## Besides the usual python packages, to do an modern analysis, we will need some additional tools.
## The most important one is the coffea package.

# - awkward
# - uproot
# - fsspec-xrootd
# - vector
# - coffea

In [2]:
import logging
import time

import awkward as ak
import cabinetry
import cloudpickle
import correctionlib
# from coffea import processor
# from coffea.nanoevents import NanoAODSchema
# from coffea.analysis_tools import PackedSelection
import copy
import hist
import matplotlib.pyplot as plt
import numpy as np
import pyhf
import requests
#import utils  # contains code for bookkeeping and cosmetics, as well as some boilerplate

logging.getLogger("cabinetry").setLevel(logging.INFO)

##
# The classics
import numpy as np
import matplotlib.pylab as plt
import matplotlib # To get the version

import pandas as pd

# The newcomers
import awkward as ak
import uproot

import vector
vector.register_awkward()

import requests
import os

import time

import json


In [3]:
import sys
sys.path.append('../')
from modules.prepare_data import *

## Choose a dataset through CMS Open Data Portal

- Access https://opendata.cern.ch/
- Choose a dataset
- Copy the link here (txt index)

In [1]:
open_data_file_id = "63168"

command = ["cernopendata-client", "get-file-locations", "--recid", open_data_file_id, "--protocol", "xrootd"]
result = subprocess.run(command, capture_output=True, text=True)
filenames = result.stdout.splitlines()
filenames_sig = np.array(filenames)

NameError: name 'subprocess' is not defined

In [6]:
# Read the file and split it into a list of root link strings

with open(f"data_txt/{file_name}", 'r', encoding='UTF-8') as file:
    root_files_list = file.read().splitlines()

root_files_list

['root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/28182352-6457-734D-995A-301A869F95B0.root',
 'root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/2C47055A-092A-3242-BEC4-6CA0B67557FA.root',
 'root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/37A146FF-19BC-4C4F-82D1-D0213E2BC798.root',
 'root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/6F892456-D392-4F43-8041-66B950DE0907.root',
 'root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/A962C9F2-A7C6-FB4B-9921-9E445A949F17.root',
 'root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/C50357CE-A9F6-EF4A-BFEB-2B6675B53C70.root']

In [14]:
## Run the loop for each file in the list

for filename in root_files_list:
    print(f"Opening...{filename}")



Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/28182352-6457-734D-995A-301A869F95B0.root
Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/2C47055A-092A-3242-BEC4-6CA0B67557FA.root
Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/37A146FF-19BC-4C4F-82D1-D0213E2BC798.root
Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/6F892456-D392-4F43-8041-66B950DE0907.root
Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/A962C9F2-A7C6-FB4B-9921-9E445A949F17.root
Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/C50357CE-A9F6-EF4A-BFEB-2B6675B53C70.root


In [97]:

## Process the files

# list_of_event_data = #['FatJet_msoftdrop', 'FatJet_particleNet_TvsQCD', 'FatJet_tau2', 'FatJet_tau3', 'FatJet_pt', 'FatJet_eta', 'FatJet_phi', 'FatJet_mass', # FatJet
# ['Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_mass', 'Muon_miniIsoId', 'Muon_tightId', # Muons
#     'Jet_btagDeepB', 'Jet_jetId', 'Jet_pt', 'Jet_eta', 'Jet_phi', 'Jet_mass', # Jets
#     'PuppiMET_pt'] # MET]

list_of_event_data = ['Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_mass', 'Muon_miniIsoId', 'Muon_tightId'] # Muons


In [76]:


try:
    f = uproot.open(root_files_list[0])
except:
    print(f"Could not open {filename}")

    
events = f['Events']

In [77]:
f.keys()

['tag;1',
 'Events;1',
 'LuminosityBlocks;1',
 'Runs;1',
 'MetaData;1',
 'ParameterSets;1']

In [113]:
selected_events = select_events(root_files_list[0], IS_DATA=False, list_of_event_data=list_of_event_data)

Opening...root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/28182352-6457-734D-995A-301A869F95B0.root
Could not open root://eospublic.cern.ch//eos/opendata/cms/Run2016G/SinglePhoton/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v2/2430000/28182352-6457-734D-995A-301A869F95B0.root


In [114]:
my_dict = {}

for col in selected_events.keys():
    #print(col)

    my_dict[col] = ak.to_dataframe(selected_events[col], levelname=['event_number', 'subevent_number'])


    print(col)
    print(len(my_dict[col]))

AttributeError: 'NoneType' object has no attribute 'keys'

In [112]:
my_dict['Muon_pt'].reset_index(level=['entry', 'subentry']).rename(columns={'entry': 'event_number', 'subentry': 'subevent_number'})

,event_number,subevent_number,values
0,2,0,22.436131
1,2,1,13.501536
2,3,0,118.352051
3,3,1,38.933064
4,6,0,3.522036
...,...,...,...
7524,28213,1,7.965006
7525,28214,0,16.564068
7526,28221,0,6.511995
7527,28222,0,3.208570


In [106]:
muon_df = pd.DataFrame.from_dict(my_dict)

In [7]:
# Download these files

for i, datasetname in enumerate(root_files_list):
    
    print(datasetname)

    outfilename = f'FILE_LIST_{datasetname}.txt'

    # Remove the file if it exists
    # try:
    #     os.remove(outfilename)
    # except OSError:
    #     pass
    url = datasetname
    #for url in root_files_list[i]:
    #print(url)

    r = requests.get(url, allow_redirects=True)

    open(outfilename, 'a').write(r.text)

root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16MiniAODv2/ZPrime2DarkPhoton_HMass-1000_DPMass-0p3_TuneCP5_13TeV-pythia8/MINIAODSIM/106X_mcRun2_asymptotic_v17-v2/30000/1EACD6D9-1EEE-B548-B47F-36E9641118C4.root


InvalidSchema: No connection adapters were found for 'root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16MiniAODv2/ZPrime2DarkPhoton_HMass-1000_DPMass-0p3_TuneCP5_13TeV-pythia8/MINIAODSIM/106X_mcRun2_asymptotic_v17-v2/30000/1EACD6D9-1EEE-B548-B47F-36E9641118C4.root'

In [ ]:
##

In [ ]:
#